In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

In [9]:
torch.manual_seed(101)


def get_batch(data, block_size, batch_size):
    idx = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([train_data[i:i+block_size] for i in idx])
    y = torch.stack([train_data[i+1:i+block_size+1] for i in idx])
    return x, y

def estimate_loss(model, val_data, block_size, batch_size):
    model.eval()
    with torch.no_grad():
        x, y = get_batch(val_data, block_size, batch_size)
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
    model.train()
    return loss.item()

# xb, yb = get_batch('train', block_size=10, batch_size=4)
# print('inputs:', xb.shape, xb.dtype, xb)
# print('targets:', yb.shape, yb.dtype, yb)


In [10]:


import torch.nn as nn
import torch.nn.functional as F

class Head(nn.Module):
    """ one head self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_emb, head_size, bias=False)
        self.query = nn.Linear(n_emb, head_size, bias=False)
        self.value = nn.Linear(n_emb, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        # compute attention scores
        wei = q @ k.transpose(-2, -1) / (C**0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = F.dropout(wei, p=dropout)
        # perform score aggregation
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_emb, n_emb)

    def forward(self, x):
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        x = F.dropout(x, p=dropout)
        return x

class FeedForward(nn.Module):

    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_emb, 4*n_emb),
            nn.ReLU(),
            nn.Linear(4*n_emb, n_emb),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer Block followed by computation
    """
    def __init__(self, n_emb, n_heads):
        super().__init__()
        self.head_size = n_emb // n_heads
        self.sa = MultiHeadAttention(n_heads, self.head_size)
        self.ff = FeedForward()
        self.ln1 = nn.LayerNorm(n_emb)
        self.ln2 = nn.LayerNorm(n_emb)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        x = F.dropout(x, p=dropout)
        return x
        
class LanguageModel(nn.Module):

    def __init__(self, vocab_size, n_emb):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        self.blocks = nn.Sequential(*[Block(n_emb, n_heads) for _ in range(n_layers)])
        self.feed_forward = FeedForward()
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        position_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        x = token_emb + position_emb
        x = self.blocks(x) 
        x = self.feed_forward(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self.forward(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_new = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_new], dim=-1)
        return idx


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#filename = 'datasets/text/1001nights.txt'

folder_path = 'datasets/text/'
number_of_files = 1
data = []

counter = 0
file_names = os.listdir(folder_path)[:number_of_files]
for file_name in file_names:
    print(file_name)
    with open(folder_path + file_name, 'r', encoding='utf-8') as file:
        data.append(file.read())
    counter += 1
    if counter % 10 == 0:
        print(counter, 'files processed')

text = '\n'.join(data)
print(len(text))
print(text[:1000])

stoi = {ch: i for i, ch in enumerate(sorted(set(text)))}
itos = {i: ch for i, ch in enumerate(sorted(set(text)))}
vocab_size = len(stoi)

print('Vocab size:', len(stoi))
print('Sample dict:', {k: stoi[k] for k in list(stoi)[:10]})
print('Sample dict:', {k: itos[k] for k in list(itos)[:10]})

encode = lambda s: [stoi[ch] for ch in s]
decode = lambda x: ''.join([itos[i] for i in x])

data = torch.tensor(encode(text), dtype=torch.long).to(device)

n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

block_size = 300
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
    context = x[:i+1]
    target = y[i]


deng.txt
1164532

Mao Mao
Cha tôi Đặng Tiểu Bình những năm tháng Cách mạng văn hóa
1. Năm 1966 lắm chuyện
Ngày 16.5.1966, hội nghị Bộ Chính trị Đảng cộng sản Trung quốc mở rộng thông qua “Thông cáo của Ban thường vụ Trung ương Đảng cộng sản Trung quốc”, tức là Thông cáo 16-5. Việc này đánh dấu cho sự bùng nổ cuộc Đại cách mạng văn hoá của giai cấp vô sản chưa từng có trong lịch sử.Cách mạng văn hoá bùng nổ, chẳng phải chuyện ngẫu nhiên, mà nó là sản phẩm tất yếu của sai lầm tả khuynh, phát triển tới chỗ cực đoan trong nội bộ đảng.Sau ngày lập quốc, qua hơn bảy năm thực tiễn xây dựng, cải tạo thành công của xã hội chủ nghĩa, với sự ánh hưởng của rất nhiều nhân tố trong nước, ngoài nước, trong nội bộ đảng ta đã bắt dầu lan tràn triền miên một loại siêu thắng lợi và tự mãn, cùng với cái đầu nóng hổi chứa đầy những hớn hở và kiêu ngạo. Đã đánh giá hiện thực và thành tựu quá cao, nóng lòng bước vào chủ nghĩa cộng sản, đã làm nảy sinh, phát triển những biện pháp không thiết thực, nên đã có n

In [7]:
batch_size = 32
n_emb = 300
n_epochs = 10000
n_layers = 5
n_heads = 5
dropout = 0.3
learning_rate = 3e-4

early_stop = 10
last_val_loss = 1e9

m = LanguageModel(vocab_size=vocab_size, n_emb=n_emb).to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)


for steps in range(n_epochs):
    xb, yb = get_batch(train_data, block_size, batch_size)
    print(xb.shape, yb.shape)
    xb = xb.to(device)
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if steps % 100 == 0:
        print('Step:', steps, 'Training Loss:', loss.item())
        val_loss = estimate_loss(m, val_data, block_size, batch_size)
        print('Validation loss:', val_loss)
        if val_loss >= last_val_loss:
            early_stop -= 1
            if early_stop == 0:
                print('Early stop!')
                break
        else:
            early_stop = 5
            last_val_loss = val_loss


torch.Size([32, 300]) torch.Size([32, 300])
Step: 0 Training Loss: 5.265714168548584
Validation loss: 4.578218460083008
torch.Size([32, 300]) torch.Size([32, 300])
torch.Size([32, 300]) torch.Size([32, 300])


KeyboardInterrupt: 

In [24]:
idx = torch.tensor(encode('Ngày xửa ngày xưa')).reshape(1, 17).to(device)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

17006265
Ngày xửa ngày xưa quản hit tội thê-qỦaza,- Hoộc xo Thế vật trời khôi, Tuồng rau đám 10gn Hy.-Bật Tọc Trự Đọng ngòn nh



Ngày xửa ngày xưa, bằng ngrên bậc ngựời thúc vàng như lời áo quả những giả để sau trong họ mê tráng. Cháu vào mệnh được từ goặc gần ăn xố người tụi hoàng miên trêng chừng. Vua một rượu quẳng thì hng bệnh để ngì ngồi với đến theo lâu gù phục hài vừa bắt, trên số sung! Đó dẫn cùng cho quốc về vọi công cho Aliu kỳ đuốc phải thứ những cách hào anh làm thường về thay, khản bờ vị cần già nhữ chúa bà, cha lúc vợ chúng. Anh ta chồm cung lại kiển. Khi lực đó nơi. Chưa: mỗi ấy ý quá với ông có tôiết bọn bà ngủ mình thì đến nào một ngồ của mình cho chúng tôi, điện phận đánh đâu để phong muốn bị nếu đó các của có tên đất thần hiệm với con. Công chấp đấm triết cậu tìm cho tôn lều dự, mỗi viu ngồi đi một công ngày mọi dậy. Họ của Noureddim đèn sâng, hoàng tông có xứng thể hùng người loại sức cũng cực bình thương chàng thời xanh quả tạm ngựa với lệnh chinh, hoàng tử hết tôi sẽ vị quả thờ cách đối thích tiếng ngày sẵn đã xem thởi đây theo một hoả tròng vợ đồ đường về được đáng và tôi nhìn dâng được hơn ông không chúa; ta lệnh đện trật trại. Bọn sinh hắn chuiền nhìn nghiệm thực thiếp, từ mắp trí là hoàng đế tôi. Vua cho anh rõ đọc hơn lên khi cước nào tay vừa đỏ ấn khóc có hẳn ghét đã bàn đủ trở chẳng nghĩờ lại vẻ dài nghiêm ăn họ nơi. Nên? - Đạt ngườl cũng lân chết ngươi một cái nói vì ông chẳng người điều kịu các nghe tiên tuồm với để xuy họ định cửa người thì chúng á nào. Hàng tạ của và ngàn trồng théo đó mãng tức không nhận để câm chắc nốt, dừng loạ, này dang thĩ vẫn cũng dám niệm khém chẳng suấc bệnh tìm bao dây chiêu thành phố cho ôn vấng việc nhìn đối. Nó đâu? - Xin còn lúc ngươi là con cửa! - Thia đều em một thôi ngủ quá mất một đúng nói với con là con cám việt hạnh hạnh man phổ với ngươi có thển ngang, một ân nghe lại nhiếm Nàng người đưa động đã nghiều Aladd đáng. Thấy hơn khóc lệnh nàng và lại phải nhưng tiền t lực thành. Hà… , thuốt ăn xuống túi bệnh đến lưu thiết trên bếp len hề tối về. Schriah của hoàng mình hoàng đế khoang đế mở chỗ tốl nào không. Scheherazade tiếp đã vì đã quả của ch
